In [1]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
[
    [[-0.3886]],

    [[-1.2214]],

    [[-0.4248]]]

In [ ]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))
for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    # print('i:                ', i)
    # print('i.view(1, 1, -1): ', i.view(1, 1, -1))
    # print('hidden: \n', hidden)
    # print()
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# Alternatively, we can do the entire sequence all at once.

# The first value returned by LSTM is all of the hidden states throughout
# the sequence. The second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:

# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time.

# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[ 0.2996,  0.2087,  0.1477]],

        [[ 0.4869,  0.0171,  0.0391]],

        [[ 0.0130,  0.3750,  0.1118]],

        [[-0.1253,  0.3174, -0.1214]],

        [[-0.0836, -0.0226, -0.1158]]], grad_fn=<MkldnnRnnLayerBackward0>)
(tensor([[[-0.0836, -0.0226, -0.1158]]], grad_fn=<StackBackward0>), tensor([[[-0.2427, -0.0349, -0.2793]]], grad_fn=<StackBackward0>))


## Example: An LSTM for Part-of-Speech Tagging

In [23]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [20]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print('\nscores are before training:\n', tag_scores)

for epoch in range(10):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        # print('sentence: ', sentence)
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()
    # print()

# See what the scores are after training
with torch.no_grad():
    test_sentence = ['The', 'dog', 'ate', 'the', 'apple']
    # tensor([[-0.9949, -0.8859, -1.5236],
    #         [-1.0763, -0.8669, -1.4318],
    #         [-1.0934, -0.8726, -1.3981],
    #         [-1.1111, -0.9105, -1.3150],
    #         [-1.0661, -0.8812, -1.4213]])
    test_sentence = ['The', 'book', 'ate', 'the', 'apple']
    
    inputs = prepare_sequence(test_sentence, word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  
    # i,j corresponds to score for tag j for word i. 
    # The predicted tag is the maximum scoring tag.

    # Here, we can see the predicted sequence below is 0 1 2 0 1 ### 1, 1, 1, 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!

    print('test sentence: ', test_sentence)
    print('\nscores are after training:\n', tag_scores)


scores are before training:
 tensor([[-1.1582, -0.9493, -1.2075],
        [-1.1234, -1.0101, -1.1691],
        [-1.0940, -1.0474, -1.1574],
        [-1.1060, -1.0406, -1.1524],
        [-1.0666, -1.1977, -1.0386]])
test sentence:  ['The', 'book', 'ate', 'the', 'apple']

scores are after training:
 tensor([[-1.1750, -0.8240, -1.3764],
        [-1.1422, -0.8838, -1.3180],
        [-1.1151, -0.9061, -1.3166],
        [-1.1256, -0.8924, -1.3248],
        [-1.1055, -0.9651, -1.2447]])
